In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import mne
from utils.loader import load_data, unicorn_fs, convert_to_mne, unicorn_eeg_channels
# %matplotlib inline
matplotlib.use("Qt5Agg")

path_recording = 'data/recordings_28_03_2024'
unicorn_channels = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8", "acc_x", "acc_y", "acc_z"]

#### Visualize an example of the data structure

1. dataframe
2. trigger
3. eeg
4. accelerations

In [5]:
path_folder = 'data/dataset/baseline'
files = [f for f in os.listdir(path_folder) if f.endswith('.csv')]
path_file = os.path.join(path_folder, files[1])
names = unicorn_eeg_channels

eeg, trigger, dataframe = load_data(path_file, header=False, fs=unicorn_fs, skiprows=0, names = names)

print(dataframe)

plt.plot(trigger)
plt.show()

raw_data = convert_to_mne(eeg, trigger, fs=unicorn_fs, chs=unicorn_eeg_channels, recompute=False) 
raw_data.plot()

# acc = np.array([dataframe.acc_x, dataframe.acc_y, dataframe.acc_z])   
# acc = acc.T 
# plt.plot(acc)
# plt.show()

           Fz      C3      Cz      C4      Pz     PO7      Oz     PO8  STI
0       0.000   1.000   2.000   3.000   4.000   5.000   6.000   7.000  NaN
1       1.749   1.248   1.213   5.307  -2.189  -3.282  -8.046  -2.180  NaN
2       1.885   3.641   1.732   9.092   1.263   0.896  -3.522  -0.331  NaN
3       4.676   7.011   2.816  13.260   6.998   6.301   3.787   4.492  NaN
4      10.856  12.777   3.051  17.522  14.289  12.315  12.471  11.509  NaN
...       ...     ...     ...     ...     ...     ...     ...     ...  ...
36246   4.907   3.516   9.581   6.595   3.688   4.655   2.970   6.656  NaN
36247   7.780   5.538  10.056   8.444   7.564   9.196   8.656  10.466  NaN
36248  10.380   8.048   8.485   9.190  10.681  13.004  13.067  12.845  NaN
36249  11.513   7.728   7.092   7.319   7.831  10.129   9.576   8.714  NaN
36250   8.731  -0.061   3.918   0.681  -2.577  -0.941  -2.450  -2.654  NaN

[36251 rows x 9 columns]
Creating RawArray with float64 data, n_channels=8, n_times=36251
    Range

C:\Users\Gianni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mne\viz\_figure.py:372: RuntimeWarning: Mean of empty slice
  data -= np.nanmean(data, axis=1, keepdims=True)


<MNEBrowseFigure size 800x749 with 4 Axes>

Channels marked as bad:
none


#### Visualize 'listening' vs 'play and listening' data

In [4]:
recording_type = ['listening','playing and listening']
music_type = 'relax' 
# music_type = 'excited'

for i in range(len(recording_type)):

    path_data = os.path.join(path_recording, recording_type[i], music_type)
    files = [f for f in os.listdir(path_data) if f.endswith('.csv')]
    path_file = os.path.join(path_data, files[0])

    # NOTE: skiprows=5 is needed to skip the first 5 rows of the csv file
    eeg, trigger, dataframe = load_data(path_file, header=False, fs=unicorn_fs, skiprows=5, names = unicorn_channels)

    # Convert to MNE format
    raw_data = convert_to_mne(eeg, trigger, fs=unicorn_fs, chs=unicorn_eeg_channels, recompute=False) 
    # recompute=True to recalculate the event labels if the values are negative

    filtered = raw_data.copy() 

    # remove power line noise
    filtered.notch_filter(50) 
    filtered.notch_filter(60) 
    # Apply band-pass filtering
    filtered.filter(1,40) 
    
    fig = filtered.plot()
    fig.suptitle('EEG - ' + recording_type[i], fontsize=16)

    pxx_filt = filtered.compute_psd(fmin=0, fmax=50)
    
    fig = pxx_filt.plot()
    fig.suptitle('Power Spectral Density - ' + recording_type[i], fontsize=16)


Creating RawArray with float64 data, n_channels=8, n_times=99934
    Range : 0 ... 99933 =      0.000 ...   399.732 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=99934
    Range : 0 ... 99933 =      0.000 ...   399.732 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal ban

Channels marked as bad:
none
Channels marked as bad:
none


#### Visualize 'baseline' vs 'play solo' data

In [3]:
recording_type = ['baseline','playing solo']

for i in range(len(recording_type)):

    path_data = os.path.join(path_recording, recording_type[i])
    files = [f for f in os.listdir(path_data) if f.endswith('.csv')]
    path_file = os.path.join(path_data, files[0])

    # NOTE: skiprows=5 is needed to skip the first 5 rows of the csv file
    eeg, trigger, dataframe = load_data(path_file, header=False, fs=unicorn_fs, skiprows=5, names = unicorn_channels)

    # Convert to MNE format
    raw_data = convert_to_mne(eeg, trigger, fs=unicorn_fs, chs=unicorn_eeg_channels, recompute=False) 
    # recompute=True to recalculate the event labels if the values are negative

    filtered = raw_data.copy() 

    # remove power line noise
    filtered.notch_filter(50) 
    filtered.notch_filter(60) 
    # Apply band-pass filtering
    filtered.filter(1,40) 

    fig1 = filtered.plot()
    fig1.suptitle('EEG - ' + recording_type[i], fontsize=16)

    pxx_filt = filtered.compute_psd(fmin=0, fmax=50)
    
    fig2 = pxx_filt.plot()
    fig2.suptitle('Power Spectral Density - ' + recording_type[i], fontsize=16)

Creating RawArray with float64 data, n_channels=8, n_times=23935
    Range : 0 ... 23934 =      0.000 ...    95.736 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=23935
    Range : 0 ... 23934 =      0.000 ...    95.736 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal ban

Channels marked as bad:
none
Channels marked as bad:
none
